In [1]:
from PIL import Image

In [6]:
import tensorflow as tf


In [26]:
img = Image.open('static-images/1.png')

In [20]:
img = Image.open('training_data/cs10000053986926948907.png')

In [99]:
input_layer = tf.keras.layers.Input(shape=(100,100, 4))
rescale = tf.keras.layers.Rescaling(1./255, input_shape=(100, 100, 4))(input_layer)
flat_layer = tf.keras.layers.Flatten()(rescale)
hidden_layer = tf.keras.layers.Dense(100, activation='sigmoid')(flat_layer)
out_later = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_layer)

model = tf.keras.Model(input_layer, out_later)
#model.build((input_size,1,1))
model.compile(
    #loss='sparse_categorical_crossentropy' ,
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"],
)


In [83]:
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 100, 100, 4)]     0         
                                                                 
 rescaling_1 (Rescaling)     (None, 100, 100, 4)       0         
                                                                 
 flatten_3 (Flatten)         (None, 40000)             0         
                                                                 
 dense_21 (Dense)            (None, 100)               4000100   
                                                                 
 dense_22 (Dense)            (None, 1)                 101       
                                                                 
Total params: 4,000,201
Trainable params: 4,000,201
Non-trainable params: 0
_________________________________________________________________


In [64]:
import numpy as np

In [69]:
img_as_array = np.array(img)

In [41]:
img_as_array

array([[[ 34,  63, 106, 255],
        [ 81,  52,  40, 255],
        [ 60,   5,  37, 255],
        ...,
        [ 11,  30,  76, 255],
        [ 86,  67,  47, 255],
        [ 85,  28,  50, 255]],

       [[ 33,  61,  84, 255],
        [ 66,  27,  40, 255],
        [ 29,  38,  29, 255],
        ...,
        [ 59,  56,  38, 255],
        [ 76,  46,  45, 255],
        [ 53,  47,  57, 255]],

       [[ 30,  81,  61, 255],
        [ 65,   0,  45, 255],
        [ 32,  77,  72, 255],
        ...,
        [ 66,  43,  59, 255],
        [ 41,  49,  52, 255],
        [ 68,  71,  42, 255]],

       ...,

       [[ 35,  73,  56, 255],
        [ 35,  25,  33, 255],
        [ 68,  47,  52, 255],
        ...,
        [ 61,  59,  47, 255],
        [ 88,  54,  15, 255],
        [ 43,  37,  75, 255]],

       [[ 51,  60,  37, 255],
        [ 52,  50,  62, 255],
        [ 49,  50,  88, 255],
        ...,
        [ 74,  74,  63, 255],
        [ 53,  21,  64, 255],
        [ 63,  51,  68, 255]],

       [[ 74

In [42]:
img_as_array == img_as_array.reshape(-1, 100, 100, 4)

array([[[[ True,  True,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True],
         ...,
         [ True,  True,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]],

        [[ True,  True,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True],
         ...,
         [ True,  True,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]],

        [[ True,  True,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True],
         ...,
         [ True,  True,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]],

        ...,

        [[ True,  True,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True],
         ...,
         [ True,  True,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  Tr

In [36]:
train_x = np.asarray(img_as_array)

In [53]:
np.array([img_as_array]).shape

(1, 100, 100, 4)

In [74]:
import pathlib

In [79]:
bad_imgs = []
for img_path in  pathlib.Path('static-images').glob('*.png'):
    bad_imgs.append(np.array(Image.open(img_path)))

In [80]:
good_imgs = []
for img_path in  pathlib.Path('training_data').glob('*.png'):
    good_imgs.append(np.array(Image.open(img_path)))

In [84]:
all_images = bad_imgs + good_imgs
all_labels = [0] * len(bad_imgs) + [1] * len(good_imgs)

In [92]:
from sklearn import model_selection

In [93]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(all_images, all_labels, test_size=.2)

In [97]:
model.predict(np.array(x_test),)

125/125 [==============================] - 1s 9ms/step


array([[1.7569766e-04],
       [9.9978030e-01],
       [1.7569681e-04],
       ...,
       [9.9978030e-01],
       [1.7569799e-04],
       [1.7569850e-04]], dtype=float32)

In [100]:
model.fit(np.array(x_train), np.array(y_train),  validation_data=(np.array(x_test), np.array(y_test)))

166/500 [========>.....................] - ETA: 11s - loss: 0.0076 - accuracy: 0.9972

In [ ]:
model.predict(x_test)